# DS105 L8 P6 Hands on Helper

In [16]:
install.packages("mvnormtest")
install.packages("car")
install.packages("plyr")


The downloaded binary packages are in
	/var/folders/9q/yp0trhm570d82rk1lfh67tq00000gn/T//RtmpDzvxQC/downloaded_packages

The downloaded binary packages are in
	/var/folders/9q/yp0trhm570d82rk1lfh67tq00000gn/T//RtmpDzvxQC/downloaded_packages

The downloaded binary packages are in
	/var/folders/9q/yp0trhm570d82rk1lfh67tq00000gn/T//RtmpDzvxQC/downloaded_packages


In [17]:
library("mvnormtest")
library("car")
library("plyr")

In [6]:
data <- read.csv("../Data/hockey1.csv")

In [9]:
head(data)

,ID,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim
,<dbl>,<int>,<int>,<chr>,<lgl>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,20120302213,2012030221,3,away,FALSE,OT,John Tortorella,2,35,44,8
2,20120302216,2012030221,6,home,TRUE,OT,Claude Julien,3,48,51,6
3,20120302223,2012030222,3,away,FALSE,REG,John Tortorella,2,37,33,11
4,20120302226,2012030222,6,home,TRUE,REG,Claude Julien,5,32,36,19
5,20120302236,2012030223,6,away,TRUE,REG,Claude Julien,2,34,28,6
6,20120302233,2012030223,3,home,FALSE,REG,John Tortorella,1,24,37,2


### independent variable is won
### dependent variables are shots and hits

In [12]:
data$shots <- as.numeric(data$shots)
data$hits <- as.numeric(data$hits)

In [13]:
data$wonFactor <- as.factor(data$won)

In [14]:
head(data)

,ID,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,wonFactor
,<dbl>,<int>,<int>,<chr>,<lgl>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<fct>
1,20120302213,2012030221,3,away,FALSE,OT,John Tortorella,2,35,44,8,FALSE
2,20120302216,2012030221,6,home,TRUE,OT,Claude Julien,3,48,51,6,TRUE
3,20120302223,2012030222,3,away,FALSE,REG,John Tortorella,2,37,33,11,FALSE
4,20120302226,2012030222,6,home,TRUE,REG,Claude Julien,5,32,36,19,TRUE
5,20120302236,2012030223,6,away,TRUE,REG,Claude Julien,2,34,28,6,TRUE
6,20120302233,2012030223,3,home,FALSE,REG,John Tortorella,1,24,37,2,FALSE


In [15]:
#feature selection
keeps <- c("wonFactor", "shots", "hits")
data <- data[keeps]

In [18]:
#assumption sample size
count(data, "wonFactor") #assumption NOT violated - more cases per number of dependent variables for male and female

wonFactor,freq
<fct>,<int>
FALSE,7441
TRUE,7441


In [21]:
#feature selection for shapiro test
keeps1 <- c("shots", "hits")
data1 <- data[keeps1]

ERROR: Error in `[.data.frame`(data[keeps1], 1:5000): undefined columns selected


In [27]:
#multivariate normality
# NOTE: we must limit this to top 5000 rows 
mshapiro.test(t(data1[1:5000,]))  #violates this assumption, p < .05


	Shapiro-Wilk normality test

data:  Z
W = 0.97779, p-value < 2.2e-16


In [29]:
#Homogeneity of Variance
leveneTest(shots ~ wonFactor, data=data) # Violates
leveneTest(hits ~ wonFactor, data=data)  # Passes

,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,1,8.369727,0.003820764
,14880,NA,NA


,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,1,3.357201,0.0669317
,14880,NA,NA


In [30]:
#Absence of Multicollinearity
cor.test(data$shots, data$hits, method="pearson", use="complete.obs") #below .7, no strong multicollinearity


	Pearson's product-moment correlation

data:  data$shots and data$hits
t = 2.2984, df = 14880, p-value = 0.02155
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.00277288 0.03489467
sample estimates:
       cor 
0.01883864 


In [37]:
#MANOVA
MANOVA <- manova(cbind(shots, hits) ~ wonFactor, data = data)

In [38]:
summary(MANOVA)

             Df    Pillai approx F num Df den Df    Pr(>F)    
wonFactor     1 0.0045269   33.831      2  14879 2.192e-15 ***
Residuals 14880                                               
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [39]:
summary.aov(MANOVA, test = "wilks") 

 Response shots :
               Df Sum Sq Mean Sq F value Pr(>F)
wonFactor       1     90  89.640  1.9297 0.1648
Residuals   14880 691208  46.452               

 Response hits :
               Df  Sum Sq Mean Sq F value    Pr(>F)    
wonFactor       1    5068  5068.5  65.269 7.032e-16 ***
Residuals   14880 1155506    77.7                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [40]:
#There is a significant difference in hits in relation to winning.